In [63]:
from shiny import App, ui, render, reactive, run_app
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from sklearn.svm import SVC
import nest_asyncio
nest_asyncio.apply()
data=None
X=None
y=None
X_train=None
X_test=None
y_train=None
y_test=None
model=None







def missing_val_impute(value):
    global data
    if value==1:
        data=data.dropna()
    if value==2:
        for i in data.columns:
            if data[i].dtype=="O":
                data[i]=data[i].fillna(data[i].mean)
    if value==3:
        for i in data.columns:
            if data[i].dtype=="O":
                data[i]=data[i].fillna(data[i].median)


def data_type_change(column_name,choice):
    global data
    if choice==1:
        data[column_name]=data[column_name].astype('int')
        
    if choice==2:
        data[column_name]=data[column_name].astype('float')
    


def encoding_fn(column_name,method):
    global data
    
    if method=="One Hot Encoding" and data[column_name].unique().size<=10:
        from sklearn.preprocessing import OneHotEncoder
        en=OneHotEncoder()
        encoded_data=en.fit_transform(data[[column_name]]).toarray()
        for i in range(encoded_data.shape[1]):
            data["En_"+str(i)]=encoded_data[:,i]
    if method=="Label Encoding" and data[column_name].unique().size<=20:
        from sklearn.preprocessing import LabelEncoder
        en=LabelEncoder()
        data["En_"+column_name]=en.fit_transform(data[column_name])

def polynomial_features(selected):
    global data
    from sklearn.preprocessing import PolynomialFeatures
    pf=PolynomialFeatures()
    if len(selected)==1:
        selected = selected[0]
        poly_data=pf.fit_transform(data[[selected]])
    else:
        selected = [col[0] if isinstance(col, tuple) else col for col in selected]
        poly_data=pf.fit_transform(data[selected])
    for i in range(poly_data.shape[1]):
        data["Poly_"+str(i)]=poly_data[:,i]

def feature_selections(feature, target):
    global data
    global X
    global y
    global X_train,X_test,y_train,y_test
    feature = [col[0] if isinstance(col, tuple) else col for col in feature]
    X = data[feature]
    y = data[target]
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test =train_test_split(X, y,test_size=0.2, random_state=42)


def model_compare_R():
    global X_train,X_test,y_train,y_test
    model=LinearRegression()
    model2=RandomForestRegressor(random_state=42)
    model=model.fit(X_train,y_train)
    model2=model2.fit(X_train,y_train)
    as1=model.score(X_test,y_test)
    as3=model2.score(X_test,y_test)
    return as1,as3


def model_compare_C():
    global X_train,X_test,y_train,y_test
    model1=LogisticRegression(random_state=42)
    model3=SVC(kernel='linear',C=1)
    model4=RandomForestClassifier(random_state=42)
    model1=model1.fit(X_train,y_train)
    model3=model3.fit(X_train,y_train)
    model4=model4.fit(X_train,y_train)
    as2=model1.score(X_test,y_test)
    as4=model3.score(X_test,y_test)
    as5=model4.score(X_test,y_test)
    return as2,as4,as5


def model_choose(selection):
    global model
    if selection=='Linear Regression':
        model=LinearRegression()

    elif selection=='Logistic Regression':
        model=LogisticRegression(random_state=42)
        
    elif selection=='Random Forest Regressor':
        model=RandomForestRegressor(random_state=42)
        
    elif selection=='Support Vector Classifier':
        model=SVC(kernel='linear',C=1)
        
    elif selection=='Random Forest Classifier':
        model=RandomForestClassifier(random_state=42)

    model=model.fit(X_train,y_train)
        


def predict(pred_features):
    
    predicted=model.predict(pred_features)
    return predicted



show=reactive.Value(False)

app_ui = ui.page_fluid(
    ui.tags.style(
        """
        h5 {
        color: blue;
        }
        h4 {
        color: green;
        }

        th, td {
            padding: 10px;  /* Add spacing */
            text-align: center; /* Center text */
        }
        h2{text-align:center;
            font-weight:bold;
            color:red;
            text-decoration:underline;
        }
        """
    ),
    ui.panel_title(" AUTO ML TOOL "),
    ui.layout_sidebar(
        ui.sidebar(
            ui.markdown("<h4><b>Preprocessing: </b></h5>"),
            ui.input_file("file","Input the CSV File",accept=[".csv"]),
            ui.input_action_button("imputer","Impute Missing Values"),
            ui.input_action_button("dtype","Change Data Type"),
            ui.input_action_button("en","Encoding Methods"),
            ui.input_action_button("poly","Create Polynomial Features"),
            ui.markdown("<h4><b>Model Building: </b></h5>"),
            ui.input_action_button("target","Feature and Target"),
            ui.input_action_button("comp","Compare Different Models"),
            ui.input_action_button("select","Select & Fit a Model"),
            ui.input_action_button("pred","Make Predictions on Input"),
        ),
        ui.output_table("data_input"),

        # ui.output_ui("imputer_ui"),
        # ui.output_ui("imputing"),
        
        # ui.output_ui("dtype_change"),
        # ui.output_ui("dtyping"),
        
        # ui.output_ui("encoder_method"),
        # ui.output_ui("encoding"),
        
        # ui.output_ui("polynomial_method"),
        # ui.output_ui("polynoming"),
        
        ui.output_ui("feature_selection"),
        ui.output_ui("featuring"),
        ui.output_ui("compare"),
        
        ui.output_ui("model_selection"),
        ui.output_ui("modeling"),
        
        ui.output_ui("predict_inputs"),
        ui.output_ui("predict_inputs2"),
        ui.output_ui("predicting"),
        
    )
)
s_val=None
def server(input,output,session):
    print("Hello")

    @output
    @render.table
    def data_input():
        global data
        path=input.file()
        if not path:
            return pd.DataFrame()
        data=pd.read_csv(path[0]["datapath"])
        return data.head()
        
    @output
    @render.ui
    @reactive.event(input.imputer)
    def imputer_ui():
        show.set(True)
        return [
            ui.markdown("<h5><b>Missing Value Imputation: </b></h5>"),
            ui.p("Enter 1 for drop, 2 for Fill Mean and 3 for Fill median"),
            ui.input_numeric("choice","Enter Choice: ",value=1,min=1,max=3),
            ui.input_action_button("start_imp","Start Imputing")
        ]
    show.set(False)
    @output
    @render.text
    @reactive.event(input.start_imp)
    def imputing():
        show.set(True)
        choice_val=input.choice()
        missing_val_impute(choice_val)
        return ui.markdown("**Missing Values Imputed Succeed**")
    
    show.set(False)
    @output
    @render.ui
    @reactive.event(input.dtype)
    
    def dtype_change():
        show.set(True)
        return [
            ui.markdown("<h5><b>Changing Feature Data Types: </b></h5>"),
            ui.p("Enter 1 for Changing to Integer and Enter 2 for Changing to Float"),
            ui.input_numeric("choice2","Enter Choice: ",value=1,min=1,max=2),
            ui.input_select("s_feature","Select a Feature: ",choices=list(data.columns)),
            ui.input_action_button("start_dtype","Start Changing Data Type")
    ]


    show.set(False)
    @output
    @render.text
    @reactive.event(input.start_dtype)
    def dtyping():
        global s_val
        show.set(True)
        s_val=input.s_feature()
        choice_val=input.choice2()
        data_type_change(s_val,choice_val)
        return ui.markdown(f"**Changed Data Type of Feature:  , {s_val}!**")


    

    show.set(False)
    @output
    @render.ui
    @reactive.event(input.en)
    
    def encoder_method():
        show.set(True)
        return [
            ui.markdown("<h5><b>Feature Encoding: </b></h5>"),
            ui.input_select("s_feature2","Select a Method: ",choices=list(["One Hot Encoding","Label Encoding"])),
            ui.input_select("s_feature3","Select a Feature: ",choices=list(data.columns)),
            ui.input_action_button("start_en","Start Encoding")
    ]


    show.set(False)
    @output
    @render.text
    @reactive.event(input.start_en)
    def encoding():
        show.set(True)
        encoding_fn(input.s_feature3(),input.s_feature2())
        e_val=input.s_feature3()
        return ui.markdown(f"**Encoding Complete: {e_val}**")





    show.set(False)
    @output
    @render.ui
    @reactive.event(input.poly)
    
    def polynomial_method():
        show.set(True)
        return [
            ui.markdown("<h5><b>Polynomial Features: </b></h5>"),
            ui.input_select("s_feature5","Select Features: ",choices=list(data.columns),multiple=True),
            ui.input_action_button("start_poly","Start Creating Features")
    ]

    show.set(False)
    @output
    @render.text
    @reactive.event(input.start_poly)
    def polynoming():
        show.set(True)
        polynomial_features(input.s_feature5())
        return ui.markdown(f"**Polynomial Features Created!**")

    show.set(False)
    @output
    @render.ui
    @reactive.event(input.target)
    def feature_selection():
        show.set(True)
        return [
            ui.markdown("<h5><b>Feature Selection: </b></h5>"),
            ui.input_select("s_feature6","Select Features (Multiple): ",choices=list(data.columns),multiple=True),
            ui.input_select("s_feature7","Select Target: ",choices=list(data.columns)),
            ui.input_action_button("start_feat","Start Creating Features")
    ]

    show.set(False)
    @output
    @render.text
    @reactive.event(input.start_feat)
    def featuring():
        show.set(True)
        feature_selections(input.s_feature6(),input.s_feature7())
        return ui.markdown(f"**Features and Target are Selected**")
        



    show.set(False)
    @output
    @render.ui
    @reactive.event(input.comp)
    def compare():
        show.set(True)
        unique=y_test.nunique()
        if (y_test.dtype == "float64" or y_test.dtype == "float32") and unique>20:
            as1,as3=model_compare_R()
            return [
                ui.markdown("<h5><b>Comparing Models: </b></h5>"),
                ui.markdown(f"*Linear Regression: {as1}*"),
                ui.markdown(f"*Random Forest Regressor: {as3}*"),
            ]
        else:
            as2,as4,as5=model_compare_C()
            return [
                ui.markdown("<h5><b>Comparing Models: </b></h5>"),
                ui.markdown(f"*Logistic Regression: {as2}*"),
                ui.markdown(f"*Support Vector Classifier: {as4}*"),
                ui.markdown(f"*Random Forest Classifier: {as5}*"),
            ]





    show.set(False)
    @output
    @render.ui
    @reactive.event(input.select)
    def model_selection():
        unique=y_test.nunique()
        if (y_test.dtype == "float64" or y_test.dtype == "float32") and unique>20:
            ls=list(['Linear Regression','Random Forest Regressor'])
        else:
            ls=list(['Logistic Regression','Support Vector Classifier','Random Forest Classifier'])
        show.set(True)
        return [
                ui.markdown("<h5><b>Model Selection: </b></h5>"),
                ui.input_select("s_feature8","Select Model: ",choices=ls),
                ui.input_action_button("start_select","Select and Fit Selected Model")
            ]

    
        

    show.set(False)
    @output
    @render.text
    @reactive.event(input.start_select)
    def modeling():
        show.set(True)
        model_choose(input.s_feature8())
        return ui.markdown(f"**Model Selected and Fitted Successfully**")


    
    show.set(False)
    @output
    @render.ui
    @reactive.event(input.pred)
    def predict_inputs():
        show.set(True)
        if X_test is not None:  # Use global X_test for feature selection
            return [ui.markdown("<h5><b>Prediction: </b></h5>"),
                [ui.input_numeric(feature, f"Enter {feature}:", value=0) for feature in X_test.columns]]

    show.set(False)
    @output
    @render.ui
    @reactive.event(input.pred)
    def predict_inputs2():
        show.set(True)
        return ui.input_action_button("start_pred", "Predict the Value")

    show.set(False)
    @output
    @render.text
    @reactive.event(input.start_pred)
    def predicting():
        show.set(True)
    
        feature_values = [input[f]() for f in X_test.columns]
    
        predicted = predict([feature_values])
        
        return ui.markdown(f"**Prediction Result: {predicted[0]}**")

        

app=App(app_ui,server)
run_app(app, port=8000, launch_browser=True)


Task exception was never retrieved
future: <Task finished name='Task-1139' coro=<Server.serve() done, defined at C:\Users\progr\AppData\Roaming\Python\Python312\site-packages\uvicorn\server.py:68> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "C:\Users\progr\AppData\Roaming\Python\Python312\site-packages\uvicorn\main.py", line 580, in run
    server.run()
  File "C:\Users\progr\AppData\Roaming\Python\Python312\site-packages\uvicorn\server.py", line 66, in run
    return asyncio.run(self.serve(sockets=sockets))
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\ProgramData\anaconda3\Lib\site-packages\nest_asyncio.py", line 30, in run
    return loop.run_until_complete(task)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\ProgramData\anaconda3\Lib\site-packages\nest_asyncio.py", line 92, in run_until_complete
    self._run_once()
  File "C:\ProgramData\anaconda3\Lib\site-packages\nest_asyncio.py", line 133, in _run_once
    handle._run()
  File 

INFO:     127.0.0.1:61897 - "GET / HTTP/1.1" 200 OK


INFO:     127.0.0.1:61898 - "WebSocket /websocket/" [accepted]
INFO:     connection open


Hello
INFO:     127.0.0.1:61897 - "POST /session/ae59ab86e6125d24837e15c6e2f28c903bc08ca470b15fc4b72d83c535d2e9b4/upload/67d21f50d50004bb34f4ea46?w= HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [23416]
